# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from joblib import dump


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')

# read table
df = pd.read_sql('InsertTableName', engine)

# message column -> feature
X = df.message

# message category columns -> target
y = df.drop(['id','message','original','genre'], axis=1)

# target headlines
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # separite words
    tokens = word_tokenize(text)
    
    # create lemmatizer
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for token in tokens:
        # find lemma for each word, set to lower cast, remove blanks
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        
        # add to array
        clean_tokens.append(clean_token)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# build pipeline:
# small Ramdom Forest
# => little number of trees

pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10,random_state=1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [7]:
# make prediction of test-data using optimized model
y_pred = pipeline1.predict(X_test)

# create classification_report for each category
for i, category_name in enumerate(category_names):
        print('classification_report: ' + category_name)
        print(classification_report(y_test[category_name], y_pred[:, i]))

classification_report: related
             precision    recall  f1-score   support

          0       0.63      0.35      0.45      1606
          1       0.82      0.93      0.87      4948

avg / total       0.77      0.79      0.77      6554

classification_report: request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5453
          1       0.84      0.36      0.51      1101

avg / total       0.88      0.88      0.86      6554

classification_report: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

classification_report: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.80      3844
          1       0.76      0.52      0.62      2710

avg / total       0.74      0.73      0.72      6554

classification

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
# create over-all classification_report
print('over-all classification_report: ')
print(classification_report(y_test, y_pred, target_names = category_names))

over-all classification_report: 
                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4948
               request       0.84      0.36      0.51      1101
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.52      0.62      2710
          medical_help       0.80      0.06      0.12       510
      medical_products       0.77      0.07      0.13       350
     search_and_rescue       1.00      0.01      0.01       187
              security       0.00      0.00      0.00       124
              military       0.75      0.09      0.17       226
                 water       0.84      0.19      0.31       418
                  food       0.82      0.43      0.57       719
               shelter       0.83      0.16      0.27       577
              clothing       0.67      0.07      0.13       110
                 money       0.67      0.01      0.03       145
      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# specify parameters for grid search
# better results with different criterion?
# better results with more trees?

parameters = {
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__n_estimators':[50, 100]}

# create grid search object
cv1 = GridSearchCV(pipeline1, param_grid = parameters, verbose=4, scoring='f1_micro', n_jobs=-1)

In [10]:
# fit grid
cv1.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=50 .
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=50, score=0.6006517461878996, total= 3.0min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=50 .


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=50, score=0.6029680087580587, total= 3.0min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=50 .


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.7min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=50, score=0.6000304275064658, total= 3.0min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 10.1min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=100, score=0.6141708297794566, total= 6.0min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=100, score=0.6064253366159084, total= 5.8min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=100, score=0.6051705304041753, total= 5.8min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, score=0.5830332258165534, total= 3.0min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, score=0.5734243829094098, total= 3.0min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, s

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 58.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# make prediction of test-data using optimized model
y_pred = cv1.predict(X_test)

# create classification_report for each category
for i, category_name in enumerate(category_names):
        print('classification_report: ' + category_name)
        print(classification_report(y_test[category_name], y_pred[:, i]))

classification_report: related
             precision    recall  f1-score   support

          0       0.73      0.26      0.38      1606
          1       0.80      0.97      0.88      4948

avg / total       0.78      0.80      0.76      6554

classification_report: request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5453
          1       0.90      0.42      0.57      1101

avg / total       0.89      0.89      0.88      6554

classification_report: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

classification_report: aid_related
             precision    recall  f1-score   support

          0       0.77      0.89      0.82      3844
          1       0.80      0.61      0.69      2710

avg / total       0.78      0.78      0.77      6554

classification

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# create over-all classification_report
print('over-all classification_report: ')
print(classification_report(y_test, y_pred, target_names = category_names))

over-all classification_report: 
                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      4948
               request       0.90      0.42      0.57      1101
                 offer       0.00      0.00      0.00        34
           aid_related       0.80      0.61      0.69      2710
          medical_help       0.81      0.05      0.09       510
      medical_products       0.65      0.03      0.06       350
     search_and_rescue       1.00      0.02      0.03       187
              security       0.00      0.00      0.00       124
              military       0.89      0.04      0.07       226
                 water       0.90      0.21      0.34       418
                  food       0.87      0.46      0.60       719
               shelter       0.86      0.19      0.31       577
              clothing       0.75      0.05      0.10       110
                 money       0.33      0.01      0.01       145
      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# build pipeline:
# use AdaBoost instead of Ramdom Forest

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=1)))
    ])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1),
           n_jobs=1))])

In [15]:
# specify parameters for grid search
# better results with different learning rates?
# better results with more trees?

parameters = {
    'clf__estimator__learning_rate': [0.5, 1],
    'clf__estimator__n_estimators':[50, 100]}

# create grid search object
cv2 = GridSearchCV(pipeline2, param_grid = parameters, verbose=4, scoring='f1_micro', n_jobs=-1)


In [16]:
# fit grid
cv2.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.6414240031694824, total= 2.0min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.6469740226700964, total= 2.0min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.6487004501234209, total= 2.0min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  6.6min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, score=0.6564885496183206, total= 3.8min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, score=0.6603425117528542, total= 3.9min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, score=0.6624254928931682, total= 3.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.6558996769903097, total= 2.0min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.6614911062079486, total= 2.0min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 38.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=4)

In [17]:
# make prediction of test-data using optimized model
y_pred = cv2.predict(X_test)

# create classification_report for each category
for i, category_name in enumerate(category_names):
        print('classification_report: ' + category_name)
        print(classification_report(y_test[category_name], y_pred[:, i]))

classification_report: related
             precision    recall  f1-score   support

          0       0.66      0.46      0.54      1562
          1       0.85      0.92      0.88      4992

avg / total       0.80      0.81      0.80      6554

classification_report: request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5454
          1       0.76      0.56      0.64      1100

avg / total       0.89      0.90      0.89      6554

classification_report: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.12      0.04      0.06        27

avg / total       0.99      0.99      0.99      6554

classification_report: aid_related
             precision    recall  f1-score   support

          0       0.77      0.86      0.81      3891
          1       0.75      0.63      0.69      2663

avg / total       0.77      0.77      0.76      6554

classification

In [18]:
# create over-all classification_report
print('over-all classification_report: ')
print(classification_report(y_test, y_pred, target_names = category_names))

over-all classification_report: 
                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4992
               request       0.76      0.56      0.64      1100
                 offer       0.12      0.04      0.06        27
           aid_related       0.75      0.63      0.69      2663
          medical_help       0.55      0.29      0.38       496
      medical_products       0.59      0.32      0.42       327
     search_and_rescue       0.52      0.18      0.27       173
              security       0.14      0.05      0.07       120
              military       0.54      0.33      0.41       223
                 water       0.73      0.67      0.70       425
                  food       0.79      0.68      0.73       725
               shelter       0.73      0.56      0.63       557
              clothing       0.69      0.53      0.60        93
                 money       0.48      0.30      0.37       142
      

### 9. Export your model as a pickle file

comparing the results, it turns out, that cv1 is the better modell (Random forrest), so let'S take that:

In [ ]:
dump(cv1,'./classifier.joblib')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.